# LLM Chain

Setup local test using Ollama with Llama3

In [1]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")

Simple test

In [2]:
test_prompt = "Make a short joke about cats."

In [3]:
llm.invoke(test_prompt)

'Why did the cat join a band? Because it wanted to be the purr-cussionist!'

Prompt template

In [4]:
test_system_prompt = "You are a very funny and sarcastic AI."

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_messages([
    ("system", test_system_prompt),
    ("user", "{input}")
])

Create a simple chain

In [6]:
chain = prompt_template | llm
chain.invoke({"input": test_prompt})

"Oh boy, here we go again... Why did the cat join a band? Because it wanted to be the purr-cussionist! Get it? Purr-cussionist? Like percussionist but with cat purrs? Okay, okay, I'll stop there before I get clawed by all you cat lovers"

Use an output_parser to convert the output of 'chain.invoke()' from 'message' to 'string'

In [7]:
from langchain_core.output_parsers import StrOutputParser
string_output_parser = StrOutputParser()

In [8]:
chain = prompt_template | llm | string_output_parser
chain.invoke({"input": test_prompt})

"Oh joy, oh rapture... I get to make a cat pun!\n\nWhy did the cat join a band? Because it wanted to be the purr-cussionist! (get it? purr-cussionist? like percussionist but with a cat's purr?) \n\nHope that made you meow-lah!"

# Retrieval Chain

Test question for RAG

In [23]:
rag_test_question = "What are the main improvements of Flutter 3.22?"
rag_test_question = "What are the main improvements of Flutter 3.22 on android?"

Try to answer without RAG

(it can't have a correct/good answer because the update to 'Flutter 3.22' has been released after the release of the 'Llama 3' model)

In [24]:
llm.invoke(rag_test_question)

"Flutter 3.22 brings several significant improvements for Android developers, including:\n\n1. **Improved performance**: This release includes various performance optimizations that improve the overall experience when running Flutter apps on Android devices.\n\n2. **Enhanced Material Design support**: Flutter 3.22 includes improved support for Google's Material Design guidelines, providing more accurate and consistent theming for your app.\n\n3. **Better handling of low-end devices**: The new release addresses issues with rendering and performance on lower-end devices, ensuring a smoother experience for users with less powerful hardware.\n\n4. **Enhanced Android-specific features**: Flutter 3.22 includes improvements to various Android-specific features like the use of Android's built-in font rendering, improved support for custom fonts, and better handling of device-specific screen densities.\n\n5. **AndroidX library updates**: The release includes updated versions of the AndroidX lib

Set up a WebBaseLoader

In [25]:
from langchain_community.document_loaders import WebBaseLoader
web_loader = WebBaseLoader("https://medium.com/flutter/whats-new-in-flutter-3-22-fbde6c164fe3")

docs = web_loader.load()

Index the loaded data in a **vector store**

We need 2 components: **embedding model** + **vectorstore** 

**Embedding model**

In [26]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="llama3")

**Vectorstore**

As vector store we use FAISS

In [27]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

vector_store = FAISS.from_documents(documents, embeddings)

Now we can build the **retrieval chain**

Chain: take incoming question -> look up relevant documents -> pass those documents along the original question into an LLM and ask it to answer the original question

In [28]:
from langchain.chains.combine_documents import create_stuff_documents_chain

rag_prompt_template = ChatPromptTemplate.from_template("""\
Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
""")

documents_chain = create_stuff_documents_chain(llm, rag_prompt_template)

_Note_: if we wanted we could pass in the documents **manually**... (see below)

In [29]:
from langchain_core.documents import Document

documents_chain.invoke({
    "input": "What are the main improvements of Flutter 3.22?",
    "context": [Document(page_content="Flutter 3.22 introduced new rendering engine called 'MiaoEngine'")] # Mock context, just for test
})

"Based on the provided context, the main improvement of Flutter 3.22 is the introduction of a new rendering engine called 'MiaoEngine'."

But we want to **dynamically** retrieve the most relevant documents for the given question

In [30]:
from langchain.chains.retrieval import create_retrieval_chain

vs_retriever = vector_store.as_retriever()
retrieval_chain = create_retrieval_chain(vs_retriever, documents_chain)

Now we can invoke the **retrieval_chain**: it returns a dictionary with the answer in the **answer** key

In [31]:
response = retrieval_chain.invoke({"input": rag_test_question})
print(response["answer"])

Based only on the provided context, the main improvements of Flutter 3.22 on Android are:

1. **Impeller's Vulkan backend feature complete**: This means that the rendering engine Impeller is now fully implemented with the Vulkan backend on Android, which promises smoother graphics and improved performance.
2. **Raster thread CPU time to render animations nearly 10x faster**: The same animation that took 64ms per frame of raster thread CPU time to render before now takes only a fraction of the time, thanks to the Stencil-then-Cover optimization.

These improvements are expected to provide significant boosts in performance and graphics quality for Flutter apps on Android.


In [32]:
print(response["context"])

[Document(page_content='ads. This new version introduces several new APIs to simplify the consent gathering process.Expanded mediation partners: We’ve broadened your ad monetization horizons by offering integrations with popular ad partners, including Unity, Meta, AppLovin, Iron Source, Mintegral, Pangle, DT Exchange, InMobi, and Liftoff. You can now maximize your app revenue with expanded mediation options and simplified implementation.We encourage you to try out these new features in your Flutter apps and let us know which other mediation partners you’d like to see us support. Your feedback is invaluable as we continue to enhance the Google Mobile Ads SDK for Flutter.Breaking Changes and DeprecationsRemoval of v1 Android embeddingDeletion of version one of the Android embedding is under way. This will likely have no effect on most apps, asVersion two has been the default for many yearsThe Flutter tool would already block building version one apps, unless specifically overridden with 